In [1]:
import networkx as nx
import math
import random
import numpy as np
import copy
from datetime import datetime

In [2]:
# g, obstacles, start_pos, goal_pos = gg.build_graph()

In [3]:
def process_obstacle(g, parent, node, goal_pos):
    neighbors = [v for v in g[node] if v != parent]
    length = len(neighbors)
    if node == goal_pos or length > 1:
        return True
    if length == 0:
        return False
        
    return process_obstacle(g, node, neighbors[0], goal_pos)

In [4]:
def move_obstacle(g, node, start_pos, goal_pos, obstacles, obstacle_poison, num_obstacle_moves, stuck_num):
    
    ## segFault
    if stuck_num == 50:
        return False, node, None
    
    possible_moves = sorted([v for v in g[node] if v != goal_pos and v != start_pos])
    #recursion
    if len(possible_moves) == 0:
        return False, node, None
        
    else:      
        move_coefficients = [obstacle_poison[i] for i in possible_moves]

        chosen_move = random.choices(possible_moves, weights=move_coefficients, k = 1)[0]
        if(chosen_move in obstacles):
            #recursion
            return move_obstacle(g, chosen_move, start_pos, goal_pos, obstacles, obstacle_poison, num_obstacle_moves, stuck_num+1)
        
        obstacles.remove(node)
        obstacles.append(chosen_move)
        num_obstacle_moves += 1

        obstacle_poison[chosen_move] *= 0.843

        return True, chosen_move, node
        # print(f'{node} -> {chosen_move}')

In [5]:
def check_goal(node, path, goal_pos):
    if node == goal_pos:
        # print("Goal!", path)
        return True

In [6]:
def saw_nodes(g, node, start_pos, saw, goal_pos, obstacles):
    
    saw_candidates = [v for v in g[node]]
    if len(saw_candidates) == 1 and node != start_pos:
        saw[node] = 0
        print(f'Sawed off: {node}')

    # Mark node if every neighbor except parent is leaf node
    count = 0
    for v in saw_candidates:
        if saw[v] == 0:
            count += 1
        elif v in obstacles and not process_obstacle(g, node, v, goal_pos):
            count += 1
            
    if count == len(saw_candidates)-1 and node != start_pos:
        saw[node] = 0

In [7]:
def traverse(
    g: nx.Graph,
    start_pos: int,
    goal_pos: int, 
    alpha: float,
    beta: float,
    end_of_the_line,
    saw,
    obstacle_poison,
    num_obstacle_moves,
    obstacles,
    num_moves,
    list_obstacle_moves
):
    path = [start_pos]
    path_weight = 0
    visited = {start_pos}
    u = start_pos
    found_goal = False
    local_obstacle_moves = copy.deepcopy(list_obstacle_moves)
    
    while len(visited) != len(g.nodes):

        ### GOAL
        found_goal = check_goal(u, path, goal_pos)
        if(found_goal):
            path_weight += num_moves[0]
            break
        ###   
        
        ### SAW
        saw_nodes(g, u, start_pos, saw, goal_pos, obstacles)  
        ###
            
        # Traverse available neighbors
        neighbors = [v for v in g[u] if v not in visited and v not in obstacles and saw[v] != 0]
        
        ### DEATH 
        if len(neighbors) == 0:
            
            obstacle_neighbors = sorted([obs for obs in g[u] if obs in obstacles])
            
            if len(obstacle_neighbors) > 0:
                random_obstacle_neighbor = random.sample(obstacle_neighbors, k = 1)[0];

                if process_obstacle(g, u, random_obstacle_neighbor, goal_pos):
                    obstacle_moved, moved_to, what_was_moved = move_obstacle(g, random_obstacle_neighbor, start_pos, goal_pos, obstacles, obstacle_poison, num_obstacle_moves, 0)
                    if obstacle_moved:
                        list_obstacle_moves.append(f'{what_was_moved}->{moved_to}')
                        local_obstacle_moves.append(f'{what_was_moved}->{moved_to}')
                        num_moves[0] += 1

            else:
                if (random.random() < 0.23):
                    random_obstacle = random.sample(obstacles, k = 1)[0]
                    obstacle_moved, moved_to, what_was_moved = move_obstacle(g, random_obstacle, start_pos, goal_pos, obstacles, obstacle_poison, num_obstacle_moves, 0)
                    if obstacle_moved:
                        list_obstacle_moves.append(f'{what_was_moved}->{moved_to}')
                        local_obstacle_moves.append(f'{what_was_moved}->{moved_to}')
                        num_moves[0] += 1
            
            if len(path) > 1:
                end_of_the_line[path[-2]] *= 0.99979
            end_of_the_line[u] *= 0.99913
            break
        ###
    
        values = ([g[u][v]['pheromones']**alpha / g[u][v]['weight']**beta * end_of_the_line[v] for v in neighbors], [v for v in neighbors])
        chosen_neighbor = random.choices(neighbors, weights=values[0], k = 1)[0]
        ## ako je prepreka, pomacinji
        
        path.append(chosen_neighbor)
        visited.add(chosen_neighbor)
        path_weight += g[u][chosen_neighbor]['weight']
        u = chosen_neighbor
        
    return path, path_weight, found_goal, local_obstacle_moves

In [8]:
from matplotlib import pyplot as plt
from copy import deepcopy

In [9]:
def aco(
    g: nx.Graph,
    start_pos: int,
    goal_pos: int,
    num_ants: int,
    rho: float,
    num_iters: int,
    theta: float,
    alpha: float,
    beta:float,
    obstacles
):
    start_time = datetime.now()
    num_moves = 0
    end_of_the_line = [1 for _ in range(len(g.nodes))]
    saw = [1 for _ in range(len(g.nodes))]
    obstacle_poison = [1 if node not in obstacles else 0.943 for node in range(len(g.nodes))]
    num_obstacle_moves = 0
    best_cycle_lens = []
    final_cycle = None
    obstacles_start = copy.deepcopy(obstacles)
    num_moves_list = [num_moves]
    best_obstacle_moves = []
    
    for i in range(num_iters):
        list_obstacle_moves = []
        cycles = [traverse(g, start_pos=start_pos, goal_pos=goal_pos, alpha=alpha, beta=beta,
                           end_of_the_line = end_of_the_line, saw = saw,
                           obstacle_poison = obstacle_poison,
                           num_obstacle_moves = num_obstacle_moves, obstacles = obstacles, num_moves=num_moves_list,
                           list_obstacle_moves = list_obstacle_moves)
                           for _ in range(num_ants)]
        
#       smanjivanje feromona
        for edge in g.edges:
            g.edges[edge]['pheromones'] *= rho

#       dodavanje feromona
        for cycle, cycle_weight, found_cycle, obstacle_moves in cycles:
            delta = theta / (cycle_weight+1)
            for u, v in zip(cycle[:-1], cycle[1:]):
                g[u][v]['pheromones'] += delta

        # lista pronadjenih puteva
        goal_cycles = [(path, path_weight, path_found, obstacle_moves) 
                       for path, path_weight, path_found, obstacle_moves in cycles if path_found]
        
        #pronadjeno resenje, izdvaja se najbolje
        if len(goal_cycles) != 0:
            best_cycle = min(goal_cycles, key=lambda c: c[1])
            best_cycle_lens.append(best_cycle[1])
            # ako je nadjeno bolje od najbolje, azuriramo
            if final_cycle is None or best_cycle[1] < final_cycle[1]:
                final_cycle = deepcopy(best_cycle)
        else:
            best_cycle_lens.append(-1)

        obstacles = copy.deepcopy(obstacles_start)
        num_obstacle_moves = 0
        # print(num_moves_list[0])
        # print(f'\nStart of iteration {i+1}\n')
        num_moves_list[0] = 0

        # if final_cycle is not None:
        #     best_cycle_lens.append(final_cycle[1])

        obstacles = copy.deepcopy(obstacles_start)
        list_obstacle_moves.clear()

    best_score_value = min(list(filter(lambda x: x != -1, best_cycle_lens)))
    total_time = (datetime.now() - start_time).total_seconds()

    solution_metrics = {
        "best_value": best_score_value, 
        "time": total_time,
        "obstacles_moved": final_cycle[3],
        "best_path": final_cycle[0],
        "solution_values": best_cycle_lens,
    }
    
    print(f'Best score: {solution_metrics["best_value"]}')
    print(f'Time it took to finish the search: {solution_metrics["time"]}')
    print(f'Obstacles moved: {solution_metrics["obstacles_moved"]}')
    print(f'Best path: {solution_metrics["best_path"]}')
    
    return solution_metrics

In [10]:
# aco(g=g, start_pos=start_pos, goal_pos=goal_pos, num_ants=30, rho=0.9, num_iters=40, alpha=0.9, beta=1.5, theta=100.0, obstacles = obstacles)